In [1]:
import time
import random
import requests
import pandas as pd
from sqlalchemy import create_engine, text

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
#!pip install psycopg2-binary

# Covid data parsing from lab01

In [3]:
# covid data parsing from lab01

resp = requests.get('https://en.wikipedia.org/wiki/COVID-19_pandemic_death_rates_by_country')

from bs4 import BeautifulSoup
soup = BeautifulSoup(resp.content, 'html.parser')
soup.title.text

'COVID-19 pandemic death rates by country - Wikipedia'

In [4]:
# Creating BeautifulSoup object
soup = BeautifulSoup(resp.text, 'html.parser')

# Verifying tables and their classes
print('Classes of each table:')
for table in soup.find_all('table'):
    print(table.get('class'))


Classes of each table:
['sidebar', 'sidebar-collapse', 'nomobile', 'nowraplinks', 'hlist']
['wikitable', 'sortable', 'sortunder', 'static-row-numbers', 'tpl-blanktable', 'plainrowheaders', 'plainrowheadersbg', 'sticky-col1']
['nowraplinks', 'hlist', 'mw-collapsible', 'expanded', 'navbox-inner']
['nowraplinks', 'mw-collapsible', 'mw-collapsed', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'mw-collapsible', 'mw-collapsed', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'mw-collapsible', 'mw-col

In [5]:
table = soup.find('table', class_='wikitable sortable sortunder static-row-numbers tpl-blanktable plainrowheaders plainrowheadersbg sticky-col1')

# Defining of the dataframe
df_covid = pd.DataFrame(columns=['Country', 'Deaths', 'Cases'])

# Collecting Ddata
for row in table.tbody.find_all('tr'):    
    # Find all data for each column
    column_country = row.find_all('th')
    column_deaths = row.find_all('td')
   
    if(column_country != [] and column_deaths != []):
        country = column_country[0].text.strip()
        deaths = column_deaths[1].text.strip()
        cases = column_deaths[2].text.strip()     
        df_covid = df_covid.append({'Country': country,  'Deaths': deaths, 'Cases': cases}, ignore_index=True)

In [6]:
df_covid.head()

,Country,Deaths,Cases
0,World[a],"6,889,730","762,200,405"
1,Peru,"219,784","4,492,891"
2,Bulgaria,"38,268","1,300,490"
3,Bosnia and Herzegovina,"16,325","402,573"
4,Hungary,"48,719","2,199,146"


## Data preprocessing

In [7]:
df_covid['Deaths'] = df_covid['Deaths'].str.replace(',', '', regex=False)
df_covid['Deaths'] = df_covid['Deaths'].str.replace('—', '0', regex=False)
df_covid['Cases'] = df_covid['Cases'].str.replace(',', '', regex=False)

In [8]:
df_covid

,Country,Deaths,Cases
0,World[a],6889730,762200405
1,Peru,219784,4492891
2,Bulgaria,38268,1300490
3,Bosnia and Herzegovina,16325,402573
4,Hungary,48719,2199146
...,...,...,...
234,Falkland Islands,0,1923
235,Pitcairn Islands,0,4
236,Tokelau,0,5
237,Niue,0,747


In [9]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239 entries, 0 to 238
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Country  239 non-null    object
 1   Deaths   239 non-null    object
 2   Cases    239 non-null    object
dtypes: object(3)
memory usage: 5.7+ KB


In [10]:
df_covid['Deaths'] = df_covid['Deaths'].astype(int)
df_covid['Cases'] = df_covid['Cases'].astype(int)

In [11]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239 entries, 0 to 238
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Country  239 non-null    object
 1   Deaths   239 non-null    int32 
 2   Cases    239 non-null    int32 
dtypes: int32(2), object(1)
memory usage: 3.9+ KB


## Features

In [12]:
df_covid['death_ratio'] = df_covid['Deaths'] / df_covid['Cases']
df_covid.head()

,Country,Deaths,Cases,death_ratio
0,World[a],6889730,762200405,0.009039
1,Peru,219784,4492891,0.048918
2,Bulgaria,38268,1300490,0.029426
3,Bosnia and Herzegovina,16325,402573,0.040552
4,Hungary,48719,2199146,0.022154


# DB connection and table creation

In [13]:
# database vars

db_name = 'alina'
db_user = 'alina'
db_pass = 'postgres'
db_host = '158.160.63.135'
db_port = '5432'

In [14]:
# database connection

db_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_user, db_pass, db_host, db_port, db_name)
db = create_engine(db_string)

In [15]:
# create table covid_deaths
df_covid.to_sql('covid_deaths', con = db,  if_exists='replace', index=False)

239

In [16]:
# let's check

query = """select * from covid_deaths"""

df = pd.read_sql_query(sql=text(query), con=db.connect())

In [17]:
df

,Country,Deaths,Cases,death_ratio
0,World[a],6889730,762200405,0.009039
1,Peru,219784,4492891,0.048918
2,Bulgaria,38268,1300490,0.029426
3,Bosnia and Herzegovina,16325,402573,0.040552
4,Hungary,48719,2199146,0.022154
...,...,...,...,...
234,Falkland Islands,0,1923,0.000000
235,Pitcairn Islands,0,4,0.000000
236,Tokelau,0,5,0.000000
237,Niue,0,747,0.000000
